## LSTM

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np  
from collections import Counter
from itertools import product

In [7]:
texts = """Bu ürün beklentimi fazlasyıla karşıladı.
Malzeme kalitesi gerçekten çok iyi.
Kargo hızlı ve sorunsuz bi şekilde elime ulaştı.
Fiyatına göre performansı harika.
Kesinlikle tavsiye ve ederim!
"""

In [8]:
words = texts.replace(".", "").replace("!", "").lower().split()

In [9]:
words

['bu',
 'ürün',
 'beklentimi',
 'fazlasyıla',
 'karşıladı',
 'malzeme',
 'kalitesi',
 'gerçekten',
 'çok',
 'iyi',
 'kargo',
 'hızlı',
 've',
 'sorunsuz',
 'bi',
 'şekilde',
 'elime',
 'ulaştı',
 'fiyatına',
 'göre',
 'performansı',
 'harika',
 'kesinlikle',
 'tavsiye',
 've',
 'ederim']

In [11]:
word_counts = Counter(words)
word_counts

Counter({'ve': 2,
         'bu': 1,
         'ürün': 1,
         'beklentimi': 1,
         'fazlasyıla': 1,
         'karşıladı': 1,
         'malzeme': 1,
         'kalitesi': 1,
         'gerçekten': 1,
         'çok': 1,
         'iyi': 1,
         'kargo': 1,
         'hızlı': 1,
         'sorunsuz': 1,
         'bi': 1,
         'şekilde': 1,
         'elime': 1,
         'ulaştı': 1,
         'fiyatına': 1,
         'göre': 1,
         'performansı': 1,
         'harika': 1,
         'kesinlikle': 1,
         'tavsiye': 1,
         'ederim': 1})

In [13]:
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
vocab

['ve',
 'bu',
 'ürün',
 'beklentimi',
 'fazlasyıla',
 'karşıladı',
 'malzeme',
 'kalitesi',
 'gerçekten',
 'çok',
 'iyi',
 'kargo',
 'hızlı',
 'sorunsuz',
 'bi',
 'şekilde',
 'elime',
 'ulaştı',
 'fiyatına',
 'göre',
 'performansı',
 'harika',
 'kesinlikle',
 'tavsiye',
 'ederim']

In [14]:
word_to_index = {word: i for i, word in enumerate(vocab)}
word_to_index

{'ve': 0,
 'bu': 1,
 'ürün': 2,
 'beklentimi': 3,
 'fazlasyıla': 4,
 'karşıladı': 5,
 'malzeme': 6,
 'kalitesi': 7,
 'gerçekten': 8,
 'çok': 9,
 'iyi': 10,
 'kargo': 11,
 'hızlı': 12,
 'sorunsuz': 13,
 'bi': 14,
 'şekilde': 15,
 'elime': 16,
 'ulaştı': 17,
 'fiyatına': 18,
 'göre': 19,
 'performansı': 20,
 'harika': 21,
 'kesinlikle': 22,
 'tavsiye': 23,
 'ederim': 24}

In [15]:
index_to_vocab = {i: word for i, word in enumerate(vocab)}
index_to_vocab

{0: 've',
 1: 'bu',
 2: 'ürün',
 3: 'beklentimi',
 4: 'fazlasyıla',
 5: 'karşıladı',
 6: 'malzeme',
 7: 'kalitesi',
 8: 'gerçekten',
 9: 'çok',
 10: 'iyi',
 11: 'kargo',
 12: 'hızlı',
 13: 'sorunsuz',
 14: 'bi',
 15: 'şekilde',
 16: 'elime',
 17: 'ulaştı',
 18: 'fiyatına',
 19: 'göre',
 20: 'performansı',
 21: 'harika',
 22: 'kesinlikle',
 23: 'tavsiye',
 24: 'ederim'}

In [16]:
data = [(words[i], words[i+1]) for i in range(len(words)-1)]
data

[('bu', 'ürün'),
 ('ürün', 'beklentimi'),
 ('beklentimi', 'fazlasyıla'),
 ('fazlasyıla', 'karşıladı'),
 ('karşıladı', 'malzeme'),
 ('malzeme', 'kalitesi'),
 ('kalitesi', 'gerçekten'),
 ('gerçekten', 'çok'),
 ('çok', 'iyi'),
 ('iyi', 'kargo'),
 ('kargo', 'hızlı'),
 ('hızlı', 've'),
 ('ve', 'sorunsuz'),
 ('sorunsuz', 'bi'),
 ('bi', 'şekilde'),
 ('şekilde', 'elime'),
 ('elime', 'ulaştı'),
 ('ulaştı', 'fiyatına'),
 ('fiyatına', 'göre'),
 ('göre', 'performansı'),
 ('performansı', 'harika'),
 ('harika', 'kesinlikle'),
 ('kesinlikle', 'tavsiye'),
 ('tavsiye', 've'),
 ('ve', 'ederim')]

In [18]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size)
        self.fc = nn.Linear(in_features=hidden_size, out_features=vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x.view(1, 1, -1))
        x = self.fc(x.view(1, -1))
        return x
    
model = LSTM(vocab_size=len(vocab), embedding_dim=1, hidden_size=1)
model

LSTM(
  (embedding): Embedding(25, 1)
  (lstm): LSTM(1, 1)
  (fc): Linear(in_features=1, out_features=25, bias=True)
)

In [19]:
def prepare_squence(seq, to_idx):
    return torch.tensor([to_idx[w] for w in seq], dtype=torch.long)


In [23]:
embedding_sizes = [8, 16]
hidden_size = [32, 64]
learning_rates = [0.001, 0.0005]

best_loss = float("inf")
best_params = {}

print("Hyperparameter tuning basliyor")

for emb_size, hidden_s, lr in product(embedding_sizes, hidden_size, learning_rates):
    print(f"Embedding dim: {emb_size}, Hidden S: {hidden_s}, Learning Rate: {lr}")

    model = LSTM(vocab_size=len(vocab), embedding_dim=emb_size, hidden_size=hidden_s)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=model.parameters(), lr=lr)

    epochs = 50
    total_loss = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for word, next_word in data:

            model.zero_grad()
            input_tensor = prepare_squence([word], word_to_index)
            target_tensor = prepare_squence([next_word], word_to_index)
            output = model(input_tensor)
            loss = loss_fn(output, target_tensor)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if epochs % 10 == 0:
            print(f"Epoch: {epoch+1}, Loss: {epoch_loss:.5f}")
        
        total_loss = epoch_loss

    if total_loss < best_loss:
        best_loss = total_loss
        best_params = {"embedding dim": emb_size, "hidden_size": hidden_s, "learning_rt":lr}

print(best_params)
print(best_loss)

Hyperparameter tuning basliyor
Embedding dim: 8, Hidden S: 32, Learning Rate: 0.001
Epoch: 1, Loss: 80.63410
Epoch: 2, Loss: 79.95472
Epoch: 3, Loss: 79.47386
Epoch: 4, Loss: 78.97199
Epoch: 5, Loss: 78.43119
Epoch: 6, Loss: 77.83633
Epoch: 7, Loss: 77.17184
Epoch: 8, Loss: 76.42155
Epoch: 9, Loss: 75.56917
Epoch: 10, Loss: 74.59889
Epoch: 11, Loss: 73.49631
Epoch: 12, Loss: 72.24942
Epoch: 13, Loss: 70.84957
Epoch: 14, Loss: 69.29217
Epoch: 15, Loss: 67.57738
Epoch: 16, Loss: 65.71045
Epoch: 17, Loss: 63.70177
Epoch: 18, Loss: 61.56672
Epoch: 19, Loss: 59.32499
Epoch: 20, Loss: 56.99963
Epoch: 21, Loss: 54.61563
Epoch: 22, Loss: 52.19852
Epoch: 23, Loss: 49.77295
Epoch: 24, Loss: 47.36170
Epoch: 25, Loss: 44.98504
Epoch: 26, Loss: 42.66042
Epoch: 27, Loss: 40.40241
Epoch: 28, Loss: 38.22282
Epoch: 29, Loss: 36.13077
Epoch: 30, Loss: 34.13284
Epoch: 31, Loss: 32.23327
Epoch: 32, Loss: 30.43419
Epoch: 33, Loss: 28.73590
Epoch: 34, Loss: 27.13718
Epoch: 35, Loss: 25.63559
Epoch: 36, Loss

In [28]:
final_model = LSTM(vocab_size=len(vocab), embedding_dim=16, hidden_size=64)
optimizer = optim.Adam(params=final_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

print("Final model training...")
epochs = 100

for epoch in range(epochs):
    epoch_loss = 0
    for word, next_word in data:

        final_model.zero_grad()
        input_tensor = prepare_squence([word], word_to_index)
        target_tensor = prepare_squence([next_word], word_to_index)
        output = final_model(input_tensor)
        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f"Final model epoch: {epoch}, loss: {epoch_loss:.5f}")

Final model training...
Final model epoch: 0, loss: 80.70260
Final model epoch: 10, loss: 61.76485
Final model epoch: 20, loss: 23.96613
Final model epoch: 30, loss: 7.86713
Final model epoch: 40, loss: 4.13841
Final model epoch: 50, loss: 2.92710
Final model epoch: 60, loss: 2.38864
Final model epoch: 70, loss: 2.10072
Final model epoch: 80, loss: 1.92783
Final model epoch: 90, loss: 1.81548


In [45]:
def predict_squence(start_word, num_of_word):
    current_word = start_word
    output_squence = [current_word]

    for _ in range(num_of_word):
        with torch.no_grad():
            input_tensor = prepare_squence([current_word], word_to_index)
            output = final_model(input_tensor)
            predicted_idx = torch.argmax(output).item()
            predicted_word = index_to_vocab[predicted_idx]
            output_squence.append(predicted_word)

            current_word = predicted_word

    return output_squence
        

In [46]:
start_word = "ürün"
num_of_word = 1

In [47]:
type(num_of_word)

int

In [48]:
predicted_sequence = predict_squence(start_word, num_of_word)
predicted_sequence

['ürün', 'beklentimi']